# Model Server Deployment Testing

## Very Important! Enter the inference url
This is the URL you got after deploying the model on OpenShift Data Science.

In [ ]:
infer_url = 'replace_me'
# Example: infer_url = 'https://my.server.address/v2/models/crops/infer'

## Requirements
This will install the requirements for the whole workshop

In [ ]:
!pip install -qr requirements.txt

## Import libraries

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from tensorflow import expand_dims
from tensorflow.keras.utils import img_to_array, load_img
from numpy import argmax, array, max as max_
import numpy as np
import requests

## Set the classes of the model
As the model will return only the id of the class, we need the corresonding array to get the full name of what was detected.

In [ ]:
class_predictions = array([
    'Corn___Common_Rust',
    'Corn___Gray_Leaf_Spot',
    'Corn___Healthy',
    'Corn___Northern_Leaf_Blight',
    'Potato___Early_Blight',
    'Potato___Healthy',
    'Potato___Late_Blight',
    'Rice___Brown_Spot',
    'Rice___Healthy',
    'Rice___Leaf_Blast',
    'Rice___Neck_Blast',
    'Wheat___Brown_Rust',
    'Wheat___Healthy',
    'Wheat___Yellow_Rust'
])

## Prepare image to test
The image is loaded, then converted into a format recognized by the API of the model server (reference: https://github.com/kserve/kserve/blob/master/docs/predict-api/v2/required_api.md#inference).

In [ ]:
img_name = 'potato_late_blight.jpg'

img = load_img(img_name, target_size=(200, 200))
img_array = img_to_array(img)
img_array = expand_dims(img_array, 0)

# json payload
img_numpy = img_array.numpy()  # Convert to numpy array
im_json = img_numpy.tolist()  # Converts the array to a nested list

data = {
        "inputs": [
            {
                "name": "input_1",
                "shape": [1, 200, 200, 3],
                "datatype": "FP32",
                "data": im_json
            }
        ]
    }

## Call the inference service

In [ ]:
response = requests.post(infer_url, json=data)
raw_output = response.json()
print(raw_output)

## Process the result

In [ ]:
arr = np.array(raw_output['outputs'][0]['data'])  # Get the response data as a NumPy Array

class_prediction = class_predictions[argmax(arr)]
score = max_(arr)
print(f'Class prediction: {class_prediction}')
model_score = round(max_(score) * 100, 2)
print(f'Model score: {model_score}')

We have verified that our inference service endpoint is working properly.

You can now get back to the instructions.